In [1]:
import pandas as pd
import nltk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer#, tfidfconverter
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import regex as re
import requests
from sklearn.svm import SVC

In [2]:
# Load data into a dataframe
addr = "/Users/tthekkum/Documents/LnD/BV/550_DataMining/Second time/week9_10/categorized-comments.jsonl"
data1 = pd.read_json(addr ,lines = True)

In [3]:
data = data1.head(1000)

In [4]:
data=data1

In [5]:
data.cat.unique()

array(['sports', 'science_and_technology', 'video_games'], dtype=object)

In [6]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/tthekkum/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tthekkum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tthekkum/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tthekkum/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
X = data['txt']
y = data['cat']

In [8]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [9]:
X1 = documents


In [10]:
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(X1).todense()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 13, stratify=y)

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((485180, 1500), (485180,), (121295, 1500), (121295,))

In [13]:
from keras.layers import Dense
from keras.models import Sequential

N_FEATURES = 1500
N_CLASSES = 3

def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(50, activation='relu'
                 , input_shape=(N_FEATURES,)
                 ))
    nn.add(Dense(15, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return nn

In [14]:
from sklearn.pipeline import Pipeline
from transformer import TextNormalizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
pipe_mnnb = Pipeline([
        #('norm', TextNormalizer()),
        #('vect', TfidfVectorizer(max_features=N_FEATURES)).todense(),
        ('nn', KerasClassifier(build_fn=build_network,
                               epochs=2,
                               batch_size=10))
    ])


In [15]:
pipe_mnnb.fit(X_train, y_train)

Epoch 1/2
48518/48518 [==============================] - 55s 1ms/step - loss: 0.4722 - accuracy: 0.8003

Pipeline(steps=[('nn',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fae538cd1d0>)])

In [17]:
py_pred1 = pipe_mnnb.predict(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,py_pred1))

[[ 2046   121  2855]
 [   74 12957 16134]
 [  648  3107 83353]]


In [19]:
print(classification_report(y_test,py_pred1))

                        precision    recall  f1-score   support

science_and_technology       0.74      0.41      0.53      5022
                sports       0.80      0.44      0.57     29165
           video_games       0.81      0.96      0.88     87108

              accuracy                           0.81    121295
             macro avg       0.78      0.60      0.66    121295
          weighted avg       0.81      0.81      0.79    121295



In [20]:
print(accuracy_score(y_test, py_pred1))



0.8108825590502494
